In [1]:
#INIZIALIZZAIZIONE SESSION E AVVIO SPARK
import pyspark as pys 
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql as sql
import pyspark.sql.functions as f
# #Spark Config (chiamare una volta sola)
conf = SparkConf().setAppName("learning RDD")
#sc = SparkContext(conf=conf) 
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

21/07/09 21:08:37 WARN Utils: Your hostname, lorenzo-X555UJ resolves to a loopback address: 127.0.1.1; using 192.168.1.141 instead (on interface wlp3s0)
21/07/09 21:08:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/07/09 21:08:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/07/09 21:08:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
#Creazione DataFrame
#creo RDD
#sc = oggetto context
sample_data = sc.parallelize([
(1, 'MacBook Pro', 2015, '15"', '16GB', '512GB SSD'
, 13.75, 9.48, 0.61, 4.02)
, (2, 'MacBook', 2016, '12"', '8GB', '256GB SSD'
, 11.04, 7.74, 0.52, 2.03)
, (3, 'MacBook Air', 2016, '13.3"', '8GB', '128GB SSD'
, 12.8, 8.94, 0.68, 2.96)
, (4, 'iMac', 2017, '27"', '64GB', '1TB SSD'
, 25.6, 8.0, 20.3, 20.8)
])

#Converto in DF
#spark = oggetto sessione 
sample_data_df = spark.createDataFrame(sample_data,[
    'Id'
, 'Model'
, 'Year'
, 'ScreenSize'
, 'RAM'
, 'HDD'
, 'W'
, 'D'
, 'H'
, 'Weight'
])

sample_data_df.take(1) #prima riga
sample_data_df.show() #tutto il DF
sample_data_df.printSchema() #schema

+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  1|MacBook Pro|2015|       15"|16GB|512GB SSD|13.75|9.48|0.61|  4.02|
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|       27"|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+

root
 |-- Id: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- Year: long (nullable = true)
 |-- ScreenSize: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- HDD: string (nullable = true)
 |-- W: double (nullable = true)
 |-- D: double (nullable = true)
 |-- H: double (nullable = true)
 |-- Weight: double (nullable = true)



In [20]:
#Vectorized UDF

import pyspark.sql.functions as f
import pandas as pd
from scipy import stats
big_df = (
spark
.range(0, 1000000)
.withColumn('val', f.rand())
)
big_df.cache()
big_df.show(3)
#UDF con decoratore (deprecato in PySpark 3. Usare hints)
@f.pandas_udf('double')
def pandas_pdf(v):
    return pd.Series(stats.norm.pdf(v))
(
big_df
.withColumn('probability', pandas_pdf(big_df.val))
.show(5)
)

+---+--------------------+
| id|                 val|
+---+--------------------+
|  0|  0.9150914763864632|
|  1|  0.9204197144347012|
|  2|0.016164751339321604|
+---+--------------------+
only showing top 3 rows

+---+--------------------+-------------------+
| id|                 val|        probability|
+---+--------------------+-------------------+
|  0|  0.9150914763864632|0.26246573513694355|
|  1|  0.9204197144347012| 0.2611854053393164|
|  2|0.016164751339321604|0.39889016215958295|
|  3| 0.37891472354166345| 0.3713067579948149|
|  4|  0.8465682773911519|  0.278795300237918|
+---+--------------------+-------------------+
only showing top 5 rows



In [4]:
#Manca esempio su introspezione NB

#Definire uno schema
import pyspark.sql.types as typ

#Definisco lo schema
sch = typ.StructType([
    typ.StructField('Id', typ.LongType(), False)
    , typ.StructField('Model', typ.StringType(), True)
    , typ.StructField('Year', typ.IntegerType(), True)
    , typ.StructField('ScreenSize', typ.StringType(), True)
    , typ.StructField('RAM', typ.StringType(), True)
    , typ.StructField('HDD', typ.StringType(), True)
    , typ.StructField('W', typ.DoubleType(), True)
    , typ.StructField('D', typ.DoubleType(), True)
    , typ.StructField('H', typ.DoubleType(), True)
    , typ.StructField('Weight', typ.DoubleType(), True)
])

sample_data_rdd = sample_data #definito prima
sample_data_rdd.take(5)

header = sample_data_rdd.first()

sample_data_rdd = (
    sample_data_rdd
    .filter(lambda row: row != header)
    .map(lambda row: (
        int(row[0])
        , row[1]
        , int(row[2])
        , row[3]
        , row[4]
        , row[5]
        , float(row[6])
        , float(row[7])
        , float(row[8])
        , float(row[9])
          )
        )
    )
sample_data_schema = spark.createDataFrame(sample_data_rdd, schema=sch)
sample_data_schema.show()


+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|       27"|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+



In [5]:
#VISTE TEMPORANEA
#sample_data_schema è richiesto
sample_data_schema.createTempView('sample_data_view')
#Meglio usare .createOrReplaceTempView(nome)

#ATTENZIONE non può creare più di una volta la stessa vista

In [19]:
#SQL sulla vista
sample_data_schema.createOrReplaceTempView('sample_data_view')

spark.sql('''
    SELECT Model, Year
    FROM sample_data_view
    WHERE Weight<10
''').show()

#Creo un nuovo DF da un RDD
models_df = sc.parallelize([
('MacBook Pro', 'Laptop')
, ('MacBook', 'Laptop')
, ('MacBook Air', 'Laptop')
, ('iMac', 'Desktop')
]).toDF(['Model','FormFactor'])

models_df.show()

#Faccio il JOIN
models_df.createOrReplaceTempView('models')

spark.sql('''
    SELECT S.*,M.FormFactor
    FROM sample_data_view AS S
        JOIN models as M ON S.Model = M.Model
    ORDER BY Weight DESC
''').show()

+-----------+----+
|      Model|Year|
+-----------+----+
|    MacBook|2016|
|MacBook Air|2016|
+-----------+----+

+-----------+----------+
|      Model|FormFactor|
+-----------+----------+
|MacBook Pro|    Laptop|
|    MacBook|    Laptop|
|MacBook Air|    Laptop|
|       iMac|   Desktop|
+-----------+----------+

+---+-----------+----+----------+----+---------+-----+----+----+------+----------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|FormFactor|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+
|  4|       iMac|2017|       27"|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|   Desktop|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|    Laptop|
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|    Laptop|
+---+-----------+----+----------+----+---------+-----+----+----+------+----------+



In [12]:
#Alcune trasformazioni

#filter
sample_data_schema.filter(sample_data_schema.Year>2015).show()

#withColumn
sample_data_schema.withColumn("HDDsplitted",f.split(f.col("HDD")," ")).show()

#summary
sample_data_schema.select("D","W").summary().show()

#freqitems
sample_data_schema.freqItems(['RAM']).show()


+---+-----------+----+----------+----+---------+-----+----+----+------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight|
+---+-----------+----+----------+----+---------+-----+----+----+------+
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|
|  4|       iMac|2017|       27"|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8|
+---+-----------+----+----------+----+---------+-----+----+----+------+

+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
| Id|      Model|Year|ScreenSize| RAM|      HDD|    W|   D|   H|Weight| HDDsplitted|
+---+-----------+----+----------+----+---------+-----+----+----+------+------------+
|  2|    MacBook|2016|       12"| 8GB|256GB SSD|11.04|7.74|0.52|  2.03|[256GB, SSD]|
|  3|MacBook Air|2016|     13.3"| 8GB|128GB SSD| 12.8|8.94|0.68|  2.96|[128GB, SSD]|
|  4|       iMac|2017|       27"|64GB|  1TB SSD| 25.6| 8.0|20.3|  20.8

In [13]:
#Alcune Azioni

#toPandas
sample_data_schema.toPandas()

,Id,Model,Year,ScreenSize,RAM,HDD,W,D,H,Weight
0,2,MacBook,2016,"12""",8GB,256GB SSD,11.04,7.74,0.52,2.03
1,3,MacBook Air,2016,"13.3""",8GB,128GB SSD,12.80,8.94,0.68,2.96
2,4,iMac,2017,"27""",64GB,1TB SSD,25.60,8.00,20.30,20.80
